In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import Constant
from tensorflow.keras import optimizers, losses
from tensorflow.keras import callbacks

from scipy.spatial.distance import pdist, squareform
from scipy import stats
from scipy.stats import norm, skew
from sklearn.decomposition import PCA

In [ ]:
print(tf.__version__)

## 1 DATA

In [ ]:
cisco = pd.read_csv('cisco.csv')
cisco['Date'] = pd.to_datetime(cisco['Date'])
cisco.head()

In [ ]:
print('Missing values:', cisco.isnull().sum().sum())
print('Data types:', cisco.dtypes.value_counts())
print('Data shape:', cisco.shape)

## 2 Pre procesing data.Statistical check

In [ ]:
sns.set(rc={'figure.figsize':(20, 10)})
sns.set_style("whitegrid")
cisco[['Cisco Systems']].plot(linewidth=0.7)

In [ ]:
sns.distplot(cisco['Cisco Systems'], fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(cisco['Cisco Systems'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('StockPrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(cisco['Cisco Systems'], plot=plt)
plt.show()

In [ ]:
d = cisco.copy(deep = True)
d.set_index('Date', inplace = True)

In [ ]:
#stock prices for 10 companies after normalization
sns.set(rc={'figure.figsize':(20, 10)})
sns.set_style("whitegrid")
d[['Cisco Systems']].plot(linewidth=0.7)

In [ ]:
#rename columns
d.columns = ['S']
d.head()

## 3 Training and Test sets for an MLP predictor

Frame a time series as a supervised learning dataset.

Arguments:

data: Sequence of observations as a list or NumPy array

n_in: Number of lag observations as input (X)

n_out: Number of observations as output (y)

dropnan: Boolean whether or not to drop rows with NaN values

Returns:
Pandas DataFrame of series framed for supervised learning.

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    #n_vars = 1 if type(data) is list else data.shape[1]
    variables = list(data.columns)
    df = data.copy(deep = True)
    cols, names = list(), list()
    #input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += ['{}(t-{})'.format(j, i) for j in variables]
    #forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += ['{}(t)'.format(j) for j in variables]
        else:
            names += ['{}(t+{})'.format(j, i) for j in variables]
  # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
means = d.loc[:,'Cisco Systems'].mean()
d.loc[:,'Cisco Systems'] = d.loc[:,'Cisco Systems']/means

In [ ]:
df = series_to_supervised(d.loc[:,'S':], n_in=19, n_out=2, dropnan=False)

In [ ]:
df.shape

In [ ]:
df.dropna(inplace = True)
df.shape

In [ ]:
df.loc[:,'MA5'] = df.loc[:,'S(t-4)':'S(t)'].mean(axis=1)
df.loc[:,'MA10'] = df.loc[:,'S(t-9)':'S(t)'].mean(axis=1)
df.loc[:,'MA20'] = df.loc[:,'S(t-19)':'S(t)'].mean(axis=1)

In [ ]:
#shape is 300 because it is all 50 stocks of t+1. Remove all except Y1(t+1) for AAPL
df.drop(columns = list(df.columns)[:5], inplace = True)
df = df[['MA5','MA10','MA20','S(t)','S(t-1)','S(t-2)','S(t-3)','S(t-4)', 'S(t-5)','S(t-6)',
         'S(t-7)', 'S(t-8)','S(t-9)', 'S(t-10)', 'S(t-11)','S(t-12)','S(t-13)','S(t-14)','S(t+1)']]
df.shape

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(df['S(t)'],label='S(t)')
plt.plot(df['MA5'],label='MA5(t)')
plt.plot(df['MA10'],label='MA10(t)')
plt.plot(df['MA20'],label='MA20(t)')
plt.legend()
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(20, 15)})
sns.set_style("whitegrid")
df[['S(t)']].plot(linewidth=0.7)

In [ ]:
sns.set(rc={'figure.figsize':(20, 15)})
sns.set_style("whitegrid")
df[['MA5']].plot(linewidth=0.7)

In [ ]:
sns.set(rc={'figure.figsize':(20, 15)})
sns.set_style("whitegrid")
df[['MA10']].plot(linewidth=0.7)

In [ ]:
sns.set(rc={'figure.figsize':(20, 15)})
sns.set_style("whitegrid")
df[['MA20']].plot(linewidth=0.7)

In [ ]:
#split into PredTest and PredTrain
train = df.sample(frac=0.9, random_state = 13)
train_idx = list(train.index)
train.shape

In [ ]:
test = df.drop(index = train_idx)
test.shape

In [ ]:
x_train, y_train = train.iloc[:,:18], train.iloc[:,18]
x_test, y_test = test.iloc[:,:18], test.iloc[:,18]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# 4 PCA for dim(H) = h

In [ ]:
corr = df.iloc[:,:18].corr()
corr.describe()

In [ ]:
e, eig_vals = np.linalg.eig(corr)
R = np.cumsum(e)/np.sum(e)
print(min(R),max(R))

In [ ]:
eigen = np.sort(e)[::-1]
N = len(eigen)

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(111)
ax.plot(np.arange(N), eigen) 
ax.set(title='Lj versus j', ylabel='Lj', xlabel='j')
plt.show()

In [ ]:
R=np.empty(N)
for i in np.arange(N):
    R[i]=sum(eigen[:i])/N
#print(R[5:])
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(111)
ax.plot(np.arange(N), R) 
ax.set(title='Rj versus j', ylabel='Rj', xlabel='j')
plt.show()

In [ ]:
for i in range(N):
    if R[i]>=0.95:
        min_r = i
        break
print(min_r, R[min_r])

In [ ]:
for i in range(N):
    if R[i]>=0.99:
        min_r = i
        break
print(min_r, R[min_r])

In [ ]:
for i in range(N):
    if R[i]>=0.995:
        min_r = i
        break
print(min_r, R[min_r])

In [ ]:
for i in range(N):
    if R[i]>=0.996:
        min_r = i
        break
print(min_r, R[min_r])

In [ ]:
#Number of weights
for i in range(N):
    if R[i]>=0.95:
        min_r = i
        break
w = 18*h + h + h + 1
print('w =',w)

## 5 MLP predictor

In [ ]:
xn_tr = x_train.shape[0]
xn_ts = x_test.shape[0]
yn_tr = y_train.shape[0]
yn_ts = y_test.shape[0]
h=2 # same model will be recontrcucted with h=3,4,10

In [ ]:
y_train = np.reshape(y_train.values,(yn_tr, 1))
y_test = np.reshape(y_test.values,(yn_ts, 1))

In [ ]:
mlp = Sequential()
mlp.add(Dense(h, activation='relu', input_dim=18, bias_initializer='glorot_normal'))
mlp.add(Dense(1, activation='relu', bias_initializer='glorot_normal'))
mlp.summary()

In [ ]:
mlp.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
class mlpMyHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.MSEtrain = []
        self.MSEtest = []
    def on_batch_end(self, batch, logs={}):
        self.MSEtrain.append(self.model.evaluate(x_train,y_train,verbose = 0))
        self.MSEtest.append(self.model.evaluate(x_test,y_test,verbose = 0))

mlpMyMonitor = mlpMyHistory()

es = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50, restore_best_weights=True)

In [ ]:
mlpMonitor = mlp.fit(x_train, y_train, epochs=200, batch_size=32, callbacks = [mlpMyMonitor, es], 
                     validation_data = (x_test, y_test), verbose = 2)

In [ ]:
mlpmsetrain = mlpMyMonitor.MSEtrain
mlpmsetest = mlpMyMonitor.MSEtest
Rmsetr = np.sqrt(mlpmsetrain)
Rmsets = np.sqrt(mlpmsetest)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(Rmsetr, color='blue', label='Train')
plt.plot(Rmsets, color='green', label='Test')
plt.legend()
plt.grid(True)

In [ ]:
pl = pd.DataFrame(mlpMonitor.history)
pl.plot(figsize=(15,8))
plt.grid(True)

In [ ]:
predtr = mlp.predict(x_train)
predts = mlp.predict(x_test)

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(y_train, color='blue', label='True value')
plt.plot(predtr, color='green', label='Predicted value')
plt.legend()
plt.grid(True)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(y_test, color='blue', label='True value')
plt.plot(predts, color='green', label='Predicted value')
plt.legend()
plt.grid(True)

MREP

In [ ]:
from scipy.stats import t
MREP_train = np.mean(np.abs(predtr-y_train)/y_train)
MREP_test = np.mean(np.abs(predts-y_test)/y_test)

sd_train = np.std(np.abs(predtr-y_train)/y_train)
sd_test = np.std(np.abs(predts-y_test)/y_test)

CI_train = MREP_train + np.array([-1,1])*t.ppf(1.95/2,df=888-1)*sd_train/np.sqrt(888)
CI_test = MREP_test + np.array([-1,1])*t.ppf(1.95/2,df=99-1)*sd_test/np.sqrt(99)

print('MREP(train) = ',MREP_train)
print('MREP(train) 95% CI: ',CI_train)

print('MREP(test) = ',MREP_test)
print('MREP(test) 95% CI: ',CI_test)

## 6 Hidden layer activity

In [ ]:
htrain = mlp.layers[0](x_train.values).numpy()

In [ ]:
Yj = np.mean(htrain, axis = 0)

In [ ]:
Wj=mlp.layers[0].get_weights()[1]

In [ ]:
IMP = Yj*Wj

## 7 Input Features

In [ ]:
INPut = x_train.values
INP = pd.DataFrame(INPut.mean(axis=0))

In [ ]:
U = mlp.layers[0].get_weights()[0]

In [ ]:
F = pd.DataFrame(np.abs(INP.reshape(18,1)*U))

In [ ]:
Larg_imp = F.iloc[:,0].nlargest(5)
Larg_imp